## Data Screening and Processing
    Project: Capstone
    Author:  Benedikt Graf
    Version: 11-22-2021

### Import Packages abd Setup Environemnt

In [ ]:
import pandas as pd                     # for data handling
import numpy as np                      # for numerical methods and data structures
import latex                            # mathematical expressions  
import re                               # regular expressions
import matplotlib.pyplot as plt         # for plotting
from matplotlib import font_manager     # for advanced fonts

In [ ]:
%%javascript

window.scroll_flag = true
window.scroll_exit = false
window.scroll_delay = 100

$(".output_scroll").each(function() {
    $(this)[0].scrollTop = $(this)[0].scrollHeight;
});

function callScrollToBottom() {
    setTimeout(scrollToBottom, window.scroll_delay);
}

function scrollToBottom() {
    if (window.scroll_exit) {
        return;
    }
    if (!window.scroll_flag) {
        callScrollToBottom();
        return;
    };
    
    $(".output_scroll").each(function() {
        if (!$(this).attr('scroll_checkbox')){
            window.scroll_flag = true;
            $(this).attr('scroll_checkbox',true);
            var div = document.createElement('div');
            var checkbox = document.createElement('input');
            checkbox.type = "checkbox";
            checkbox.onclick = function(){window.scroll_flag = checkbox.checked}
            checkbox.checked = "checked"
            div.append("Auto-Scroll-To-Bottom: ");
            div.append(checkbox);
            $(this).parent().before(div);
        }
        
        $(this)[0].scrollTop = $(this)[0].scrollHeight;
    });
    callScrollToBottom();
}
scrollToBottom(); 


### Load Raw Data

In [ ]:
data = pd.read_stata("./data/tocodeparish/placestocodeparish.dta")
# Save a copy of orginal birthplace variable
data["og_ns_birtphlace"] = data["ns_birthplace"]


In [ ]:
data.shape


In [ ]:
# defining missing values
data["birthcountry"] = data["birthcountry"].replace({'': np.NaN, ' ': np.NaN, 'Unknown': np.NaN, "Sw":"Sweden"})
data["birthcounty"] = data["birthcounty"].replace({'': np.NaN, ' ': np.NaN, 'NA': np.NaN, 'Unknown': np.NaN})
data["ns_birthplace"] = data["ns_birthplace"].replace({'': np.NaN, ' ': np.NaN, 'NA': np.NaN, 'Unknown': np.NaN})
data["birthplace"] = data["birthplace"].replace({'': np.NaN, ' ': np.NaN, 'NA': np.NaN, 'Unknown': np.NaN})


### Describe Raw Data (Table 1, a)

In [ ]:
# measure length of strings
data['length']  = data['ns_birthplace'].str.len()


In [ ]:
# susbet to non-zero length strings
plot_data = data[(data['length'] != 0)]
plot_data['length'].describe()

### Pre-Processing 

In [ ]:
# fill missing values in ns_birthplace column with birthplace value
data["ns_birthplace"].fillna(data["birthplace"], inplace=True)


In [ ]:
data["birthcountry"] = data["birthcountry"].replace({np.NaN : ''})
data["birthcounty"] = data["birthcounty"].replace({np.NaN : ''})
data["ns_birthplace"] = data["ns_birthplace"].replace({np.NaN : ''})
data["birthplace"] = data["birthplace"].replace({np.NaN : ''})


In [ ]:
by_country = data.groupby(by = 'birthcountry').count()
by_country
pd.DataFrame(by_country["fsid"].sort_values(ascending=False,)).head(5)


In [ ]:
#energy['Country'].str.replace(r'[...]', np.nan)
#data[data["ns_birthplace"].str.contains("Nederkalix")]


In [ ]:
data[data["ns_birthplace"].str.contains("Kopparberg")].sample(10)

#### Replace Punctuation, Common Abbreviations

In [ ]:
# replace some strings

### Punctuation ###
data["ns_birthplace"] = data["ns_birthplace"].str.replace(',', ' ')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('/', ' ')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('-', ' ')
data["ns_birthplace"] = data["ns_birthplace"].str.replace(';', ' ')
data["ns_birthplace"] = data["ns_birthplace"].str.replace(':', '')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('.', '')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('<', '')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('>', '')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('(', '')
data["ns_birthplace"] = data["ns_birthplace"].str.replace(')', '')
data["ns_birthplace"] = data["ns_birthplace"].str.replace(' Lan ', '')
data["ns_birthplace"] = data["ns_birthplace"].str.replace(' län ', '')


### Sweden ###
data["ns_birthplace"] = data["ns_birthplace"].str.replace('SWEDEN', 'Sweden')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('SVERIGE', 'Sverige')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('SUECIA', 'Sweden')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Schweden', 'Sweden')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('SCHWEDEN', 'Sweden')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Suède', 'Sweden')

### Counties ###

## Älvsborg
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Alvsbr', 'Älvsborg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Alvsbg', 'Älvsborg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Alvsborg', 'Älvsborg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace(' Älv ', 'Älvsborg')

## Blekinge
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Blkng', 'Blekinge')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Blkg', 'Blekinge')

## Dalarna
mask = (data.birthcounty == "Dalarna")
data.loc[mask, 'ns_birthplace'] =  data.loc[mask, 'ns_birthplace'].str.replace('Kopparberg', 'Dalarna')

## Gävleborg
data["ns_birthplace"] = data["ns_birthplace"].str.replace('GAVLBG', 'Gävleborg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('GAVLEBORG', 'Gävleborg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Gavleborg', 'Gävleborg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Gvleborg', 'Gävleborg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Gavlbg', 'Gävleborg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Gvlbrg', 'Gävleborg')

## Göteborg och Bohus
data["ns_birthplace"] = data["ns_birthplace"].str.replace('G O Bh', 'Göteborg och Bohus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('G O BH', 'Göteborg och Bohus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('G o Bh', 'Göteborg och Bohus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('G Bhs', 'Göteborg och Bohus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('G&BH', 'Göteborg och Bohus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Got & Bh', 'Göteborg och Bohus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Got & B', 'Göteborg och Bohus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Gotebg Bohus', 'Göteborg och Bohus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Goteborg o Bohus', 'Göteborg och Bohus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Goteborg O Bohus', 'Göteborg och Bohus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Goteborg Bohus', 'Göteborg och Bohus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Goteborg och Bohus', 'Göteborg och Bohus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Göteborgs And Bohus', 'Göteborg och Bohus')

# Halland
data["ns_birthplace"] = data["ns_birthplace"].str.replace('HALLAND', 'Halland')

## Jämtland
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Jamtland', 'Jämtland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Jmtlnd', 'Jämtland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Jamtld', 'Jämtland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Jmtld', 'Jämtland')

## Jönköping
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Jonkoping', 'Jönköping')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Jnkping', 'Jönköping')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Jonkpg', 'Jönköping')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Jnkpng', 'Jönköping')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Jknpng', 'Jönköping')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Jkpg', 'Jönköping')

## Kalmar
data["ns_birthplace"] = data["ns_birthplace"].str.replace('KALMAR', 'Kalmar')

## Kronoberg
data["ns_birthplace"] = data["ns_birthplace"].str.replace('KRONOBERG', 'Kronoberg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Krnbrg', 'Kronoberg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Kronbg', 'Kronoberg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Kronobg', 'Kronoberg')

## Skaraborg
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Skrbrg', 'Skaraborg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Skarabg', 'Skaraborg')

## Skåne
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Skane', 'Skåne')
data["ns_birthplace"] = data["ns_birthplace"].str.replace(' Skne ', 'Skåne')

## Småland
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Smaland', 'Småland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('SMALAND', 'Småland')

## Södermanland
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Sadermanland', 'Södermanland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Sodmld', 'Södermanland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Sdrmnl', 'Södermanland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('S Manld', 'Södermanland')

## Stockholm
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Stklm', 'Stockholm')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Sthlm', 'Stockholm')

# Malmöhus
data["ns_birthplace"] = data["ns_birthplace"].str.replace('MALMOHUS', 'Malmöhus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Malmohus', 'Malmöhus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Malmhs', 'Malmöhus')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Mlmhs', 'Malmöhus')

## Norrbotten
data["ns_birthplace"] = data["ns_birthplace"].str.replace('NORRBOTTEN', 'Norrbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Nrrbtt', 'Norrbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Nrrbtn', 'Norrbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('N bottn', 'Norrbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace(' Norrb ', 'Norrbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('N Bottn', 'Norrbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Nbtn', 'Norrbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Nrrb', 'Norrbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Nrb', 'Norrbotten')

## Östergötland
data["ns_birthplace"] = data["ns_birthplace"].str.replace('OSTERGOTLAND', 'Östergötland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Ostergotland', 'Östergötland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Ostrgt', 'Östergötland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Ogtld', 'Östergötland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('O Gotld', 'Östergötland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('O Gtld', 'Östergötland')

## Örebro
data["ns_birthplace"] = data["ns_birthplace"].str.replace(' Orebro ', 'Örebro')
data["ns_birthplace"] = data["ns_birthplace"].str.replace(' ÖRE ', 'Örebro')

## Västerbotten
data["ns_birthplace"] = data["ns_birthplace"].str.replace('VASTERBOTTEN', 'Västerbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vasterbotten', 'Västerbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vsterbotten', 'Västerbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vesterbotten', 'Västerbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vstrbt', 'Västerbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vbtn', 'Västerbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vstb', 'Västerbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vbottn', 'Västerbotten')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('V bott', 'Västerbotten')

## Västernorrland
data["ns_birthplace"] = data["ns_birthplace"].str.replace('VÄSTERNORRLAND', 'Västernorrland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('VASTERNORRLAND', 'Västernorrland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vasternorrland', 'Västernorrland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vsternorrland', 'Västernorrland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vstrnr', 'Västernorrland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vnorld', 'Västernorrland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('V Norrld', 'Västernorrland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('V Norld', 'Västernorrland')

## Västmanland
data["ns_birthplace"] = data["ns_birthplace"].str.replace('VASTMANLAND', 'Västmanland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vastmanland', 'Västmanland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vstmanland', 'Västmanland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vstmnl', 'Västmanland')

## Värmland
data["ns_birthplace"] = data["ns_birthplace"].str.replace('VARMLAND', 'Värmland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Varmland', 'Värmland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Varmld', 'Värmland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vrmland', 'Värmland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Vrmlnd', 'Värmland')

## Gotland
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Gtlnd', 'Gotland')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Gtln', 'Gotland')


### Miscellaneous ###
data["ns_birthplace"] = data["ns_birthplace"].str.replace('FARILA', 'Farila')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('MALMÖ', 'Malmö')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('GOTHENBURG', 'Gothenburg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('KOPPARBERG', 'Kopparberg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('KRISTIANSTAD', 'Kristianstad')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('KPPBR', 'Kopparberg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('KPPRBR', 'Kopparberg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Kpprbr', 'Kopparberg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Koppbg', 'Kopparberg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Kppbrg', 'Kopparberg')
data["ns_birthplace"] = data["ns_birthplace"].str.replace('Kopparbergs', 'Kopparberg')

#### Restrict Sample to Sweden

In [ ]:
to_query = data[(data['birthcountry'] == 'Sweden') | (data["birthcountry"] == "")].reset_index(drop=True)
total_post = to_query.shape[0]
# drop duplicate values
to_query = to_query.drop_duplicates(subset=['ns_birthplace']).reset_index(drop = True)

#### Loop to Split Observations by Word Capitalization

In [ ]:
for i in range(0,len(to_query)):
    #if " " not in (to_query['ns_birthplace'].loc[i]):
 
        # split strings to list by capitlization
    s = to_query['ns_birthplace'].loc[i]
        #print(s)
    
    res = False
    
    for ele in s:
        if ele.isupper():
            res = True
            break
           
    if res == True:
    # checking for uppercase character and flagging
        
        pos = [j for j,e in enumerate(s+'A') if e.isupper()]
        list_ = [s[pos[k]:pos[k+1]] for k in range(len(pos)-1)]
        
        # reform stings that were all caps as single words
        word = ""
        for a in list_:
            if a.isupper():
                word += str(a)
                word = word.title()
        
        if word != "":
            counter = 0
            for b in list_:
                if counter < 1:
                    if b.isupper():
                        list_[list_.index(b)] = word
                        counter += 1

            for c in list_[:]:
                print(c)
                if c.isupper():
                    list_.remove(c)
               
        # update last word to Sweden
#        if (len(list_) > 1):
#            if ((list_[-1] == "Swed") or (list_[-1] == "Swd") or 
#                (list_[-1] == "Swe") or (list_[-1] == "Swdn")):
#                list_[-1] = "Sweden"
        
        #if " " not in (to_query['ns_birthplace'].loc[i]):
        string = " ".join(list_)
        res = " ".join(string.split())
        #else:
        #    string = "".join(list_)
            
        to_query['ns_birthplace'].loc[i] = res
            
        print(s, "→", to_query['ns_birthplace'].loc[i])
        

#### Loop to Standardize Names

In [ ]:
matches = ["Sweden", "Sverige", "United States"]

for i in range(0,len(to_query)):
    
    s = to_query['ns_birthplace'].loc[i]
    s_list = s.split()
    
    # some of the trickier renames
    for j in range(0,len(s_list)):
        
        if s_list[j] == "Upps":
            s_list[j] = "Uppsala"
        
        if s_list[j] == "Gotl":
            s_list[j] = "Gotland"
            
        if s_list[j] == "Varm":
            s_list[j] = "Värmland"
            
        if s_list[j] == "Soder":
            s_list[j] = "Södermanland"

        if s_list[j] == "Jnk":
            s_list[j] = "Jönköping"
                        
        if s_list[j] == "Ble":
            s_list[j] = "Blekinge"
            
        if s_list[j] == "Blek":
            s_list[j] = "Blekinge"   
    
    if (len(s_list) > 1):
        if ((s_list[-1] == "Swed") or (s_list[-1] == "Swd") or 
            (s_list[-1] == "Swe") or (s_list[-1] == "Swdn")):
            #print(s_list)
            s_list[-1] = "Sweden"
            #print(s_list)
            
      # append "Sweden" if not in

    string = " ".join(s_list)
    to_query['ns_birthplace'].loc[i] = string
    
    if not any(x in (to_query['ns_birthplace'].loc[i]) for x in matches):
        to_query['ns_birthplace'].loc[i] += str(" Sweden")
        
    print(s, "→", to_query['ns_birthplace'].loc[i]) 
    

In [ ]:
#to_query[to_query["ns_birthplace"].str.contains(" Swed ")]


In [ ]:
#to_query.to_csv("full_cleaned_data.csv")


### Save and Describe "Cleaned" Data (Table 1, b)

In [ ]:
to_query = to_query.drop_duplicates(subset=['ns_birthplace']).reset_index(drop = True)
to_query['length']  = to_query['ns_birthplace'].str.len()
#to_query.to_csv("./data/tocodeparish/cleaned_data.csv")
to_query['length'].describe()


In [ ]:
to_query[to_query['birthcountry'] == ""]


### Table of Observations (Table 2)
This includes missing birthplaces

In [ ]:
unique_bplace_post = to_query.shape[0]
unique_countries_post = to_query.drop_duplicates(subset=['birthcountry']).shape[0]
unique_counties_post = to_query.drop_duplicates(subset=['birthcounty']).shape[0]


In [ ]:
total_pre = data.shape[0]
unique_bplace_pre = data.drop_duplicates(subset=['ns_birthplace']).shape[0]
unique_countries_pre = data.drop_duplicates(subset=['birthcountry']).shape[0]
unique_counties_pre = data.drop_duplicates(subset=['birthcounty']).shape[0]


In [ ]:
meta = pd.DataFrame(columns=["Pre", "Post"], data=[[total_pre, total_post],
                                                   [unique_bplace_pre, unique_bplace_post],
                                                   [unique_countries_pre, unique_countries_post],
                                                   [unique_counties_pre, unique_counties_post]])

In [ ]:
meta
# includes 1 missing birthcountry and county in each column